In [1]:
!nvidia-smi

Tue Sep 26 12:37:40 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0     Off |                  N/A |
|  0%   36C    P8    14W / 200W |      0MiB /  8113MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:03:00.0     Off |                  N/A |
|  0%   34C    P8    15W / 200W |      0MiB /  8112MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
import tensorflow as tf
import numpy as np
import sys
import os

sys.path.append(os.path.join("..", "..", ".."))
import ecg.dataset as ds
from ecg.batch import ModelEcgBatch

%env CUDA_VISIBLE_DEVICES=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
config.gpu_options.allow_growth=True

Using TensorFlow backend.


env: CUDA_VISIBLE_DEVICES=0


In [3]:
fft_train_pipeline = (ds.Pipeline()
                      .init_variable("loss", init=list)
                      .load(fmt="wfdb", components=["signal", "meta"])
                      .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv",
                            fmt="csv", components="target")
                      .drop_labels(["~"])
                      .replace_labels({"N": "NO", "O": "NO"})
                      .random_resample_signals("normal", loc=300, scale=10)
                      .drop_short_signals(4000)
                      .segment_signals(3000, 3000)
                      .binarize_labels()
                      .signal_transpose([0, 2, 1])
                      .ravel()
                      .train_on_batch('fft_inception', metrics=f1_score, average='macro')
                      .run(batch_size=300, shuffle=True,
                           drop_last=True, n_epochs=1, prefetch=0, lazy=True))

In [4]:
index = ds.FilesIndex(path="/notebooks/data/ECG/training2017/*.hea", no_ext=True, sort=True)
eds = ds.Dataset(index, batch_class=ModelEcgBatch)
eds.cv_split(0.8)
res = (eds.test >> fft_train_pipeline).run()

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
res.get_variable('loss')

[0.69373888, 0.69205081, 0.69044971, 0.68812984, 0.68632752]

In [6]:
res.get_model_by_name('fft_inception').hist

{'train_loss': [0.69373888, 0.69205081, 0.69044971, 0.68812984, 0.68632752],
 'train_metric': [0.47539975399753998,
  0.47764095917044719,
  0.47332421340629272,
  0.47590759075907596,
  0.46508563899868249],
 'val_loss': [],
 'val_metric': []}

In [7]:
fft_test_pipeline = (ds.Pipeline()
                     .init_variable("loss", init=list)
                     .load(fmt="wfdb", components=["signal", "meta"])
                     .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv",
                           fmt="csv", components="target")
                     .drop_labels(["~"])
                     .replace_labels({"N": "NO", "O": "NO"})
                     .random_resample_signals("normal", loc=300, scale=10)
                     .drop_short_signals(4000)
                     .segment_signals(3000, 3000)
                     .binarize_labels()
                     .signal_transpose([0, 2, 1])
                     .ravel()
                     .test_on_batch('fft_inception', metrics=f1_score, average='macro')
                     .run(batch_size=300, shuffle=True, drop_last=True,
                          n_epochs=1, prefetch=0, lazy=True))

In [8]:
res_test = (eds.test >> fft_test_pipeline).run()

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
fft_predict_pipeline = (ds.Pipeline()
                        .init_variable("prediction", [])
                        .load(fmt="wfdb", components=["signal", "meta"])
                        .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv", fmt="csv", components="target")
                        .drop_labels(["~"])
                        .replace_labels({"N": "NO", "O": "NO"})
                        .random_resample_signals("normal", loc=300, scale=10)
                        .drop_short_signals(4000)
                        .segment_signals(3000, 3000)
                        .binarize_labels()
                        .signal_transpose([0, 2, 1])
                        .ravel()
                        .predict_on_batch('fft_inception')
                        .run(batch_size=100, shuffle=False, drop_last=False,
                             n_epochs=1, prefetch=0, lazy=True))

In [13]:
res_predict = (eds.test >> fft_predict_pipeline).run()

In [15]:
res_predict.get_variable('prediction')[0].shape

(286, 2)